In [1]:
import cv2
import json
# from google.colab.patches import cv2_imshow
from datetime import datetime
from matplotlib import pyplot as plt
import numpy as np
import math
from PIL import Image
import os
import pandas as pd

In [2]:
#video from camera starts with 760 width 428 height
def video_to_frame_arr(video_path):
  cap = cv2.VideoCapture(video_path)
  frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  max_frame = frame_count
  output_arr = []
  # naive version (took 24s just appending frames)
  success, img = cap.read()
  while max_frame >= 0:
    max_frame = max_frame - 1
    output_arr.append(img)
    # read next frame
    success, img = cap.read()
  return output_arr

In [3]:
def get_keypoints_descriptors(image):
  # Reading the image and converting into B/W
  #image = cv2.imread(image_path)
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # Applying the function
  sift = cv2.xfeatures2d.SIFT_create()
  kp, des = sift.detectAndCompute(gray_image, None)
    
  # uncomment to draw keypoints on image
  #kp_image = cv2.drawKeypoints(image, kp, None, color=(
      #0, 255, 0)
      #, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
  
  return(kp, des)

In [4]:
def feature_matching(des, des2):
  FLANN_INDEX_KDTREE = 1
  index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
  search_params = dict(checks = 50)
  flann = cv2.FlannBasedMatcher(index_params, search_params)
  matches = flann.knnMatch(des,des2,k=2)
  # store all the good matches as per Lowe's ratio test.
  good = []
  for m,n in matches:
      if m.distance < 0.7*n.distance:
          good.append(m)
  return good

In [5]:
def get_homography_matrix(image, image2, kp, kp2, good, MIN_MATCH_COUNT):
  if len(good)>MIN_MATCH_COUNT:
      src_pts = np.float32([ kp[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
      dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
      M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
      matchesMask = mask.ravel().tolist()
      h = image.shape[0]
      w = image.shape[1]
      pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
      dst = cv2.perspectiveTransform(pts,M)
      #image2_lines = cv2.polylines(image2,[np.int32(dst)],True,255,3, cv2.LINE_AA)
  else:
      print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
      matchesMask = None
      M = []
  return M
#uncomment this if you want to circle the RANSAC inliers/outliers and how they connect between the images
"""
  draw_params = dict(matchColor = (0,255,0), # draw matches in green 
                    singlePointColor = None,
                    matchesMask = matchesMask, # draw only inliers
                    flags = cv2.DrawMatchesFlags_DEFAULT) #flags was 2
  image3 = cv2.drawMatches(image,kp,image2,kp2,good,None,**draw_params)
"""

'\n  draw_params = dict(matchColor = (0,255,0), # draw matches in green \n                    singlePointColor = None,\n                    matchesMask = matchesMask, # draw only inliers\n                    flags = cv2.DrawMatchesFlags_DEFAULT) #flags was 2\n  image3 = cv2.drawMatches(image,kp,image2,kp2,good,None,**draw_params)\n'

In [6]:
#this is the same as get_homography_matrix but it also returns a side by side of the two images with RANSAC inliers/outliers circled and the inliers connected between images
def get_homography_matrix_old(image, image2, kp, kp2, good, MIN_MATCH_COUNT):
  if len(good)>MIN_MATCH_COUNT:
      src_pts = np.float32([ kp[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
      dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
      M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
      matchesMask = mask.ravel().tolist()
      h = image.shape[0]
      w = image.shape[1]
      pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
      dst = cv2.perspectiveTransform(pts,M)
      #image2_lines = cv2.polylines(image2,[np.int32(dst)],True,255,3, cv2.LINE_AA)
  else:
      print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
      matchesMask = None
      M = []
  draw_params = dict(matchColor = (0,255,0), # draw matches in green 
                    singlePointColor = None,
                    matchesMask = matchesMask, # draw only inliers
                    flags = cv2.DrawMatchesFlags_DEFAULT) #flags was 2
  image3 = cv2.drawMatches(image,kp,image2,kp2,good,None,**draw_params)
  return M, image3

In [7]:
def warpTwoImages(img1, img2, H):
    '''warp img2 to img1 with homography matrix H'''
    h1,w1 = img1.shape[:2]
    h2,w2 = img2.shape[:2]
    pts1 = np.float32([[0,0],[0,h1],[w1,h1],[w1,0]]).reshape(-1,1,2)
    pts2 = np.float32([[0,0],[0,h2],[w2,h2],[w2,0]]).reshape(-1,1,2)
    pts2_ = cv2.perspectiveTransform(pts2, H)
    pts = np.concatenate((pts1, pts2_), axis=0)
    [xmin, ymin] = np.int32(pts.min(axis=0).ravel() - 0.5)
    [xmax, ymax] = np.int32(pts.max(axis=0).ravel() + 0.5)
    t = [-xmin,-ymin]
    Ht = np.array([[1,0,t[0]],[0,1,t[1]],[0,0,1]]) # translate

    result = cv2.warpPerspective(img2, Ht.dot(H), (xmax-xmin, ymax-ymin))
    result[t[1]:h1+t[1],t[0]:w1+t[0]] = img1
    return result

In [8]:
#save array of frames to video 
def save_to_video(output_frame_arr, fps):
  height,width,layers=output_frame_arr[0].shape
  video=cv2.VideoWriter(filename = '/content/interval_five_later.avi',fourcc = 0,fps = fps,frameSize = (760, 428))
  for j in range(len(output_frame_arr)):
    video.write(output_frame_arr[j])
  video.release()

In [9]:
#resize all frames in an array to the same resolution (specify desired width and height as parameters)
def resize_all(pano_frames_arr, width, height):
  resized_pano_arr = []
  height,width,layers=pano_frames_arr[0].shape
  for i in range(len(pano_frames_arr)):
    resized_pano_arr.append(cv2.resize(pano_frames_arr[i], (width, height)))
  return resized_pano_arr

In [10]:
#given two frames, this function warps them to the same plane and translates them to their proper position on a shared background by adding padding where needed
#You need to use this if you don't want the final panorama to be cropped to the resolution of one of the original frames
def warpPerspectivePadded(src, dst, transf):

    src_h, src_w = src.shape[:2]
    lin_homg_pts = np.array([[0, src_w, src_w, 0], [0, 0, src_h, src_h], [1, 1, 1, 1]])

    trans_lin_homg_pts = transf.dot(lin_homg_pts)
    trans_lin_homg_pts /= trans_lin_homg_pts[2,:]

    minX = np.min(trans_lin_homg_pts[0,:])
    minY = np.min(trans_lin_homg_pts[1,:])
    maxX = np.max(trans_lin_homg_pts[0,:])
    maxY = np.max(trans_lin_homg_pts[1,:])

    # calculate the needed padding and create a blank image to place dst within
    dst_sz = list(dst.shape)
    pad_sz = dst_sz.copy() # to get the same number of channels
    pad_sz[0] = np.round(np.maximum(dst_sz[0], maxY) - np.minimum(0, minY)).astype(int)
    pad_sz[1] = np.round(np.maximum(dst_sz[1], maxX) - np.minimum(0, minX)).astype(int)
    dst_pad = np.zeros(pad_sz, dtype=np.uint8)

    # add translation to the transformation matrix to shift to positive values
    anchorX, anchorY = 0, 0
    transl_transf = np.eye(3,3)
    if minX < 0: 
        anchorX = np.round(-minX).astype(int)
        transl_transf[0,2] += anchorX
    if minY < 0:
        anchorY = np.round(-minY).astype(int)
        transl_transf[1,2] += anchorY
    new_transf = transl_transf.dot(transf)
    new_transf /= new_transf[2,2]
    print(new_transf)

    dst_pad[anchorY:anchorY+dst_sz[0], anchorX:anchorX+dst_sz[1]] = dst
    dest_pad_pre_warp = dst_pad

    warped = cv2.warpPerspective(src, new_transf, (pad_sz[1],pad_sz[0]), borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))

    return dst_pad, warped, dest_pad_pre_warp

In [11]:
#warps a single point from one plane to another given a homography matrix M
def warp_point(x, y, M):
    d = M[2][0] * x + M[2][1] * y + M[2][2]

    return (
        int((M[0, 0] * x + M[0, 1] * y + M[0, 2]) / d), # x
        int((M[1, 0] * x + M[1, 1] * y + M[1, 2]) / d), # y
    )

In [12]:
#this is like warpPerspectivePadded but for N frames instead of 2
def warp_n_with_padding(dst, src_list, transf_list, main_frame_arr):
  #main_frame_arr = main_frame_arr2
  #dst = main_frame_arr[505]
  #src_list = [main_frame_arr[450], main_frame_arr[480], main_frame_arr[508], main_frame_arr[512], main_frame_arr[525]]
  #transf_list = [hm13, hm23, hm43, hm53, hm63]

  pad_sz_0_arr = []
  pad_sz_1_arr = []
  minMaxXY_arr = []
  dst_sz = list(dst.shape)

  for i in range(len(src_list)):
    src_h, src_w = src_list[i].shape[:2]
    lin_homg_pts = np.array([[0, src_w, src_w, 0], [0, 0, src_h, src_h], [1, 1, 1, 1]])
    trans_lin_homg_pts = transf_list[i].dot(lin_homg_pts)
    trans_lin_homg_pts /= trans_lin_homg_pts[2,:]

    minX = np.min(trans_lin_homg_pts[0,:])
    minY = np.min(trans_lin_homg_pts[1,:])
    maxX = np.max(trans_lin_homg_pts[0,:])
    maxY = np.max(trans_lin_homg_pts[1,:])

    pad_sz0 = np.round(np.maximum(dst_sz[0], maxY) - np.minimum(0, minY)).astype(int)
    pad_sz1 = np.round(np.maximum(dst_sz[1], maxX) - np.minimum(0, minX)).astype(int)

    minMaxXY_arr.append([minX, minY, maxX, maxY])
    pad_sz_0_arr.append(pad_sz0)
    pad_sz_1_arr.append(pad_sz1)

  # calculate the needed padding and create a blank image to place dst within
  pad_sz = dst_sz.copy() # to get the same number of channels
  pad_sz[0] = max(pad_sz_0_arr)
  pad_sz[1] = max(pad_sz_1_arr)
  indexY = pad_sz_0_arr.index(pad_sz[0])
  indexX = pad_sz_1_arr.index(pad_sz[1])
  minY = minMaxXY_arr[indexY][1]
  maxY = minMaxXY_arr[indexY][3]
  minX = minMaxXY_arr[indexX][0]
  maxX = minMaxXY_arr[indexX][2]
  dst_pad = np.zeros(pad_sz, dtype=np.uint8)

  #add translation to ALL transformation matrices to shift to positive values
  new_transf_list = []
  anchorX_list = []
  anchorY_list = []
  for i in range(len(transf_list)):
    anchorX, anchorY = 0, 0
    transl_transf = np.eye(3,3)
    if minX < 0: 
        anchorX = np.round(-minX).astype(int)
        transl_transf[0,2] += anchorX
    if minY < 0:
        anchorY = np.round(-minY).astype(int)
        transl_transf[1,2] += anchorY
    new_transf = transl_transf.dot(transf_list[i])
    new_transf /= new_transf[2,2]
    new_transf_list.append(new_transf)
    anchorX_list.append(anchorX)
    anchorY_list.append(anchorY)

  anchorX = max(anchorX_list)
  anchorY = max(anchorY_list)
  dst_pad[anchorY:anchorY+dst_sz[0], anchorX:anchorX+dst_sz[1]] = dst

  warped_src_arr = []
  for i in range(len(src_list)):
    warped = cv2.warpPerspective(src_list[i], new_transf_list[i], (pad_sz[1],pad_sz[0]), borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
    warped_src_arr.append(warped)
  
  return dst_pad, warped_src_arr, new_transf_list, anchorX, anchorY

In [13]:
#converts all the warped + translated pieces of the panorama from RGB to RGBA images (you'll need the alpha channel [which sets opacity] for compositing)
def get_rgba_im_arr(dst_par, warped_src_arr):  
  im_arr = []
  im = Image.fromarray(dst_pad)
  im = im.convert("RGBA")
  im = np.asarray(im)
  im_arr.append(im)
  for i in range(len(warped_src_arr)):
    im2 = Image.fromarray(warped_src_arr[i])
    im2 = im2.convert("RGBA")
    im2 = np.asarray(im2)
    im_arr.append(im2)
  
  return im_arr

In [14]:
#converts an array containing an RGB image to an array containing an RGBA image (adds alpha channel)
def rgba_to_rgb(comp_arr):
  im = Image.fromarray((comp_arr).astype(np.uint8))
  im = im.convert('RGB')
  im = np.asarray(im)
  return im

In [15]:
#this function is like. the slowest possible way to do this. Will be updating soon
def alpha_composite_n_images(im_arr):
  #naive solution
  comp = []
  im = im_arr[0]
  for row in range(im.shape[0]):
    comp_inner = []
    for col in range(im.shape[1]):
      #figure out which images are black at this pixel
      not_black_list = []
      black_list = []
      for i in range(len(im_arr)):
        if im_arr[i][row][col][0] == 0 and im_arr[i][row][col][1] == 0 and im_arr[i][row][col][2] == 0:
          black_list.append(im_arr[i])
        else:
          not_black_list.append(im_arr[i])
      #if all images are black, set to transparent
      if len(not_black_list) == 0:
        comp_inner.append([0, 0, 0, 0])
      #if only one image is NOT black, use it
      elif len(not_black_list) == 1:
        comp_inner.append(not_black_list[0][row][col])
      #if multiple images are not black, alpha blend them all together
      else:
        alpha = 1/len(not_black_list)
        channel1 = 0
        channel2 = 0
        channel3 = 0
        for j in range(len(not_black_list)):
            channel1 = channel1 + alpha * not_black_list[j][row][col][0]
            channel2 = channel2 + alpha * not_black_list[j][row][col][1]
            channel3 = channel3 + alpha * not_black_list[j][row][col][2]
        comp_inner.append([channel1, channel2, channel3, 255])
    comp.append(comp_inner)
  comp_arr = np.array(comp)
  return comp_arr

"""
 #if all images are black, set to black
      if len(not_black_list) == 0:
        comp_inner.append([0, 0, 0, 255])
      """

'\n #if all images are black, set to black\n      if len(not_black_list) == 0:\n        comp_inner.append([0, 0, 0, 255])\n      '

In [16]:
def alpha_composite_two(im, im2):
  comp = []
  #np.zeros((im.shape[0], im.shape[1], im.shape[2]))
  alpha = 0.5
  for row in range(im.shape[0]):
    comp_inner = []
    for col in range(im.shape[1]):
      #if one image is black, just use the other
      if (im[row][col][0] == 0 and im[row][col][1] == 0 and im[row][col][2] == 0) and (im2[row][col][0] != 0 or im2[row][col][1] != 0 or im2[row][col][2] != 0):
        comp_inner.append(im2[row][col])
      elif (im2[row][col][0] == 0 and im2[row][col][1] == 0 and im2[row][col][2] == 0) and (im[row][col][0] != 0 or im[row][col][1] != 0 or im[row][col][2] != 0):
        comp_inner.append(im[row][col])
      #if both images are black, set to transparent
      elif (im[row][col][0] == 0 and im[row][col][1] == 0 and im[row][col][2] == 0) and (im2[row][col][0] == 0 and im2[row][col][1] == 0 and im2[row][col][2] == 0):
        comp_inner.append([0, 0, 0, 0])
      #if both pixels are not black, alpha blend
      else:
        channel1 = alpha * im[row][col][0] + (1 - alpha) * im2[row][col][0]
        channel2 = alpha * im[row][col][1] + (1 - alpha) * im2[row][col][1]
        channel3 = alpha * im[row][col][2] + (1 - alpha) * im2[row][col][2]
        comp_inner.append([channel1, channel2, channel3, 255])
    comp.append(comp_inner)
  return comp

In [17]:
#warps panorama back to rectangle after compositing. Doesn't work for every case yet
#this version works by tracing where the corners of the original frames are warped to in the panorama and using those to "pull" it back into a rectangle
def warp_back_to_rect_up(og_src, org_dst, final_width, final_height, anchorX, anchorY, hm_og_src_og_dst, comp_arr):
  og_dst_width = org_dst.shape[1]
  og_dst_height =org_dst.shape[0]
  og_dst_corners = [[anchorX, anchorY], [anchorX + og_dst_width, anchorY], [anchorX + og_dst_width, anchorY + og_dst_height], [anchorX, anchorY + og_dst_height]] #clockwise from top left

  og_src_width = og_src.shape[1]
  og_src_height = og_src.shape[0]
  og_src_warped_top_left = warp_point(0, 0, hm_og_src_og_dst)
  og_src_warped_top_right = warp_point(og_src_width, 0, hm_og_src_og_dst)
  og_src_warped_bottom_right = warp_point(og_src_width, og_src_height, hm_og_src_og_dst)
  og_src_warped_bottom_left = warp_point(0, og_src_height, hm_og_src_og_dst)
  og_src_warped_corners = [og_src_warped_top_left, og_src_warped_top_right, og_src_warped_bottom_right, og_src_warped_bottom_left]

  src_quad_list = np.float32([og_dst_corners[0], og_dst_corners[1], og_src_warped_corners[2], og_src_warped_corners[3]])
  dst_quad_list = np.float32([[0, 0], [final_width, 0], [final_width, final_height], [0, final_height]])

  homography_matrix = cv2.getPerspectiveTransform(src_quad_list, dst_quad_list)

  rect = cv2.warpPerspective(comp_arr, homography_matrix, (760, 428))

  for j in range(rect.shape[0] - 1):
    for i in range(len(rect[0])):
      if rect[j][i][3] == 255:
        top_left_x = i
        break

  for j in range(rect.shape[0] - 1):
    for i in range(len(rect[0])):
      if rect[j][(len(rect[0]) - 1) - i][3] == 255:
        top_right_x = i
        break

  for j in range(rect.shape[0] - 1):
    for i in range(rect.shape[1] - 1):
      if rect[(rect.shape[0] - 1) - j][i][3] == 255:
        bottom_left_x = i
        break

  for j in range(rect.shape[0] - 1):
    for i in range(len(rect[rect.shape[0] - 1])):
      if rect[(rect.shape[0] - 1) - j][(rect.shape[0] - 1) - i][3] == 255:
        bottom_right_x = i
        break

  for j in range(rect.shape[1] - 1):
    for i in range(len(rect[:,0])):
      if rect[i][j][3] == 255:
        top_left_y = i
        break

  for j in range(rect.shape[1] - 1):
    for i in range(rect.shape[0]):
      if rect[i][(rect.shape[1] - 1) - j][3] == 255:
        top_right_y = i
        break

  for j in range(rect.shape[1] - 1):
    for i in range(rect.shape[0]):
      if rect[(rect.shape[0] - 1) - i][(rect.shape[1] - 1) - j][3] == 255:
        bottom_right_y = i
        break

  for j in range(rect.shape[1] - 1):
    for i in range(len(rect[:,0])):
      if rect[(rect.shape[0] - 1) - i][j][3] == 255:
        bottom_left_y = i
        break

  #crop rectangle using where src_quad_list warped to as 4 corners 
  top_bound = max(top_left_y, top_right_y)
  bottom_bound = rect.shape[0] - min(bottom_left_y, bottom_right_y)
  left_bound = max(top_left_x, bottom_left_x)
  right_bound = rect.shape[1] - min(top_right_x, bottom_right_x)

  del_row_arr = []
  #rows/y
  for i in range(rect.shape[0]):
    if i < top_bound or i > bottom_bound:
      del_row_arr.append(i)
  rect = np.delete(rect, del_row_arr, 0)

  del_col_arr = []
  #cols/x
  for j in range(rect.shape[1]):
    if j < left_bound or j > right_bound:
      del_col_arr.append(j)
  rect = np.delete(rect, del_col_arr, 1)

  rect = cv2.resize(rect, (760, 428))

  return rect, homography_matrix

In [18]:
#crops any fully transparent rows or columns off an image
def crop_transparent(comp_arr):
  col_sums = np.sum(comp_arr, axis = 0)
  del_cols_list = np.where(col_sums == 0)[0]
  comp_arr = np.delete(comp_arr, del_cols_list, 1)
  row_sums = np.sum(comp_arr, axis = 1)
  del_rows_list = np.where(row_sums == 0)[0]
  comp_arr = np.delete(comp_arr, del_rows_list, 0)
  return comp_arr

In [19]:
#this is a helper function used for matching the video frames to object detection outputs
# find element closest to given target using binary search.
 
def findClosest(arr, n, target):
 
    # Corner cases
    if (target <= arr[0]):
        return arr[0]
    if (target >= arr[n - 1]):
        return arr[n - 1]
 
    # Doing binary search
    i = 0; j = n; mid = 0
    while (i < j):
        mid = (i + j) // 2
 
        if (arr[mid] == target):
            return arr[mid]
 
        # If target is less than array
        # element, then search in left
        if (target < arr[mid]) :
 
            # If target is greater than previous
            # to mid, return closest of two
            if (mid > 0 and target > arr[mid - 1]):
                return getClosest(arr[mid - 1], arr[mid], target)
 
            # Repeat for left half
            j = mid
         
        # If target is greater than mid
        else :
            if (mid < n - 1 and target < arr[mid + 1]):
                return getClosest(arr[mid], arr[mid + 1], target)
                 
            # update i
            i = mid + 1
         
    # Only single element left after search
    return arr[mid]
 
 
# Method to compare which one is the more close.
# We find the closest by taking the difference
# between the target and both values. It assumes
# that val2 is greater than val1 and target lies
# between these two.
def getClosest(val1, val2, target):
 
    if (target - val1 >= val2 - target):
        return val2
    else:
        return val1

In [20]:
def get_sec_from_start(current_timestamp, start_timestamp):
  sec_from_start = (float(current_timestamp[0:-2]) - float(start_timestamp[0:-2]))/1000
  return sec_from_start

In [21]:
def warp_one_bbox(x, y, w, h, M):
  
  #warp top left corner 
  new_x, new_y = warp_point(x, y, M)

  #warp bottom right corner
  new_w, new_h = warp_point(w, h, M)

  return new_x, new_y, new_w, new_h

In [22]:
#gets the x,y coords of how much the destination image was translated during the warping process
def get_anchors(dst, src_list, transf_list, main_frame_arr):
  pad_sz_0_arr = []
  pad_sz_1_arr = []
  minMaxXY_arr = []
  dst_sz = list(dst.shape)

  for i in range(len(src_list)):
    src_h, src_w = src_list[i].shape[:2]
    lin_homg_pts = np.array([[0, src_w, src_w, 0], [0, 0, src_h, src_h], [1, 1, 1, 1]])
    trans_lin_homg_pts = transf_list[i].dot(lin_homg_pts)
    trans_lin_homg_pts /= trans_lin_homg_pts[2,:]

    minX = np.min(trans_lin_homg_pts[0,:])
    minY = np.min(trans_lin_homg_pts[1,:])
    maxX = np.max(trans_lin_homg_pts[0,:])
    maxY = np.max(trans_lin_homg_pts[1,:])

    pad_sz0 = np.round(np.maximum(dst_sz[0], maxY) - np.minimum(0, minY)).astype(int)
    pad_sz1 = np.round(np.maximum(dst_sz[1], maxX) - np.minimum(0, minX)).astype(int)

    minMaxXY_arr.append([minX, minY, maxX, maxY])
    pad_sz_0_arr.append(pad_sz0)
    pad_sz_1_arr.append(pad_sz1)

  # calculate the needed padding and create a blank image to place dst within
  pad_sz = dst_sz.copy() # to get the same number of channels
  pad_sz[0] = max(pad_sz_0_arr)
  pad_sz[1] = max(pad_sz_1_arr)
  indexY = pad_sz_0_arr.index(pad_sz[0])
  indexX = pad_sz_1_arr.index(pad_sz[1])
  minY = minMaxXY_arr[indexY][1]
  maxY = minMaxXY_arr[indexY][3]
  minX = minMaxXY_arr[indexX][0]
  maxX = minMaxXY_arr[indexX][2]
  #dst_pad = np.zeros(pad_sz, dtype=np.uint8)

  #add translation to ALL transformation matrices to shift to positive values
  new_transf_list = []
  anchorX_list = []
  anchorY_list = []
  for i in range(len(transf_list)):
    anchorX, anchorY = 0, 0
    transl_transf = np.eye(3,3)
    if minX < 0: 
        anchorX = np.round(-minX).astype(int)
        transl_transf[0,2] += anchorX
    if minY < 0:
        anchorY = np.round(-minY).astype(int)
        transl_transf[1,2] += anchorY
    new_transf = transl_transf.dot(transf_list[i])
    new_transf /= new_transf[2,2]
    new_transf_list.append(new_transf)
    anchorX_list.append(anchorX)
    anchorY_list.append(anchorY)

  anchorX = max(anchorX_list)
  anchorY = max(anchorY_list)
  #dst_pad[anchorY:anchorY+dst_sz[0], anchorX:anchorX+dst_sz[1]] = dst

  #warped_src_arr = []
  #for i in range(len(src_list)):
    #warped = cv2.warpPerspective(src_list[i], new_transf_list[i], (pad_sz[1],pad_sz[0]), borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
    #warped_src_arr.append(warped)
  
  return anchorX, anchorY

END OF FUNCTIONS

In [30]:
# from google.colab import drive
# drive.mount('/content/gdrive',force_remount=True)

# vars for jupyter notebook
cwd = os.getcwd()  # current local dir
input_dir = cwd + "/data/"
output_dir = cwd + "/results/"

test_videos = [input_dir+"avis/"+file for file in os.listdir(input_dir+"avis/") if file[-4:]==".avi"]
print("Found Videos:\n", '\n '.join(test_videos))

test_boxes = [input_dir+"object_detection/"+file for file in os.listdir(input_dir+"object_detection/") if file[-5:]==".json"]
print("Found Boxes:\n", '\n '.join(test_boxes))

chunks_to_process = 5  # 85

Found Videos:
 /Users/korahughes/Documents/GitHub/panoramic-mosaics/data/avis/2023.04.11-17.25.01-main.avi
 /Users/korahughes/Documents/GitHub/panoramic-mosaics/data/avis/2023.03.20-19.05.44-main.avi
 /Users/korahughes/Documents/GitHub/panoramic-mosaics/data/avis/2023.03.29-17.39.21-main.avi
Found Boxes:
 /Users/korahughes/Documents/GitHub/panoramic-mosaics/data/object_detection/2023.04.11-17.25.01-detic:image.json
 /Users/korahughes/Documents/GitHub/panoramic-mosaics/data/object_detection/2023.03.20-19.05.44-detic:image.json
 /Users/korahughes/Documents/GitHub/panoramic-mosaics/data/object_detection/2023.03.29-17.39.21-detic:image.json


2023.03.29-17.39.21 stats (these should be the same for every video but it's always good to check):

*  video is 15 fps
*  bboxes are 1.69 fps

CODE TO GENERATE PANORAMAS

In [27]:
#I don't know why but openCV gets mad if the video is in any format but AVI. I use this to convert mp4 to avi: https://cloudconvert.com/mp4-to-avi
main_frame_arr = video_to_frame_arr(test_videos[2])

In [29]:
#each time this takes 0, 6, 12, 18, 24, 30, 36, 42, 47th indices from current window
#windows are currently discrete chunks of the array (doing one for each timestep was taking too long to process but it's the same idea)
main_frame_arr2 = main_frame_arr
pano_frames = []
start_index = 0
start_chunk = 0
done = 0
#filename_count = 0
indices = [12, 18, 24, 30, 36, 42, 47]
#these three arrays are just grabbing the homography matrices I was trying to use to warp the bounding boxes they're not needed to make the panorama itself
hm12_pre_warp_n_with_padding_arr = []
hm12_post_warp_n_with_padding_arr = []
hm_rect_arr = []

for i in range(chunks_to_process): #this is for this specific example, but it's range(number of chunks you're diving the video into, where each chunk becomes one panorama)
  print("processing chunk", str(i)+"...")
  hm12_pre_warp_n_with_padding_arr_inner = []
  hm12_post_warp_n_with_padding_arr_inner = []
  hm_rect_arr_inner = [] 

  start_chunk = 47* i + start_index + (47 * done)
  window_frame1 = main_frame_arr2[start_chunk + 0]
  window_frame2 = main_frame_arr2[start_chunk + 6]
  kp1, des1 =get_keypoints_descriptors(window_frame1)
  kp2, des2 = get_keypoints_descriptors(window_frame2)
  matches12 = feature_matching(des1, des2)
  hm12 = get_homography_matrix(window_frame1, window_frame2, kp1, kp2, matches12, 4)
  hm12_pre_warp_n_with_padding_arr_inner.append(hm12)

  if len(hm12) == 0:
    comp_arr1 = window_frame2
  else:
    #NEED TO USE THE NEW HM FROM HERE
    dst_pad, warped_src_arr, new_transf_list, anchorX, anchorY = warp_n_with_padding(window_frame2, [window_frame1], [hm12], main_frame_arr2)
    hm12_post_warp_n_with_padding_arr_inner.append(new_transf_list[0])
    im_arr1 = get_rgba_im_arr(dst_pad, warped_src_arr)
    comp_arr1 = alpha_composite_n_images(im_arr1)

  #ROUNDS 2-6
  for j in range(5):
    new_frame = main_frame_arr2[start_chunk + indices[j]]
    rgb_comp_arr1 = rgba_to_rgb(comp_arr1)
    kp1, des1 = get_keypoints_descriptors(rgb_comp_arr1)
    kp2, des2 = get_keypoints_descriptors(new_frame)
    matches12 = feature_matching(des1, des2)
    hm12 = get_homography_matrix(rgb_comp_arr1, new_frame, kp1, kp2, matches12, 4)

    if len(hm12) == 0:
      continue
    else:
      dst_pad, warped_src_arr, new_transf_list, anchorX, anchorY = warp_n_with_padding(new_frame, [rgb_comp_arr1], [hm12], main_frame_arr2)
      im_arr1 = get_rgba_im_arr(dst_pad, warped_src_arr)
      comp_arr1 = alpha_composite_n_images(im_arr1)

  cv2.imwrite("/content/case_study_pre_rect_panos/" + str(done) + ".png", comp_arr1)
  rect1, homography_matrix_rect = warp_back_to_rect_up(rgb_comp_arr1, new_frame, 760, 428, anchorX, anchorY, hm12, comp_arr1)
  hm_rect_arr_inner.append(homography_matrix_rect)

  rect1 = crop_transparent(rect1)
  pano_frames.append(rect1)
  cv2.imwrite("/content/case_study_panos_manual/" + str(done) + ".png", rect1)
  #filename_count = filename_count + 1
  print(start_chunk)

processing chunk 0...
Not enough matches are found - 1/4


error: OpenCV(4.6.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/core/src/matmul.dispatch.cpp:550: error: (-215:Assertion failed) scn + 1 == m.cols in function 'perspectiveTransform'


### Adding Bounding Boxes

In [ ]:
vidcap = cv2.VideoCapture(test_videos[2])
fps = vidcap.get(cv2.CAP_PROP_FPS)

print(f"{fps} frames per second")

with open(test_boxes[2], 'r') as j:
     detic_dict = json.loads(j.read())

In [ ]:
#gets the # of seconds from start for each frame
frame_sfs_arr = []
sfs = 0
for i in range(len(main_frame_arr)):
  frame_sfs_arr.append(sfs)
  sfs = sfs + (1/15)
    
#have to match frames with timestamps (each frame needs to have a timestamp)
timestamps_sfs_arr = []
start_timestamp = detic_dict[0]["timestamp"]
for i in range(len(detic_dict)):
  current_timestamp = detic_dict[i]["timestamp"]
  sec_from_start = get_sec_from_start(current_timestamp, start_timestamp)
  timestamps_sfs_arr.append(sec_from_start)

In [ ]:
frames_timestamps_arr = []  # TODO: find out what this is doing
arr = timestamps_sfs_arr
n = len(arr)
for i in range(len(main_frame_arr)):
  target = frame_sfs_arr[i]
  closest = findClosest(arr, n, target)
  target_idx = timestamps_sfs_arr.index(closest)
  timestamp = detic_dict[target_idx]["timestamp"]
  frames_timestamps_arr.append(timestamp)

In [ ]:
""" Bounding Box / time
- at every time step, draw all bounding boxes from the beginning of the vis to now
- rgba : a inversely proportional to (distance from the present)^2
-- play around with color, thickness & how much of the box is being drawn (whole box vs. picture-frame vs. just corners)
"""
# time represented by blue-red color scale: stolen from d3.interpolateOrRd - https://observablehq.com/@d3/color-schemes
OrRd = ["#fff7ec","#fff7eb","#fff6ea","#fff6e9","#fff5e7","#fff5e6","#fff4e5","#fff4e4","#fff3e3","#fff3e2","#fff2e1","#fff2e0","#fff1de","#fff1dd","#fff0dc","#fff0db","#feefda","#feefd9","#feeed7","#feeed6","#feedd5","#feedd4","#feecd3","#feecd2","#feebd0","#feebcf","#feeace","#feeacd","#fee9cc","#fee9ca","#fee8c9","#fee8c8","#fee7c7","#fee7c6","#fee6c4","#fee5c3","#fee5c2","#fee4c1","#fee4bf","#fee3be","#fee3bd","#fee2bc","#fee1ba","#fee1b9","#fee0b8","#fee0b7","#fedfb5","#fedeb4","#fedeb3","#fdddb2","#fddcb1","#fddcaf","#fddbae","#fddaad","#fddaac","#fdd9ab","#fdd8a9","#fdd8a8","#fdd7a7","#fdd6a6","#fdd6a5","#fdd5a4","#fdd4a3","#fdd4a1","#fdd3a0","#fdd29f","#fdd29e","#fdd19d","#fdd09c","#fdcf9b","#fdcf9a","#fdce99","#fdcd98","#fdcc97","#fdcc96","#fdcb95","#fdca94","#fdc994","#fdc893","#fdc892","#fdc791","#fdc690","#fdc58f","#fdc48e","#fdc38d","#fdc28c","#fdc18b","#fdc08a","#fdbf89","#fdbe88","#fdbd87","#fdbc86","#fdbb85","#fdba84","#fdb983","#fdb882","#fdb781","#fdb680","#fdb57f","#fdb47d","#fdb27c","#fdb17b","#fdb07a","#fdaf79","#fdae78","#fdac76","#fdab75","#fdaa74","#fca873","#fca772","#fca671","#fca46f","#fca36e","#fca26d","#fca06c","#fc9f6b","#fc9e6a","#fc9c68","#fc9b67","#fb9a66","#fb9865","#fb9764","#fb9563","#fb9462","#fb9361","#fb9160","#fa905f","#fa8f5e","#fa8d5d","#fa8c5c","#f98b5b","#f9895a","#f98859","#f98759","#f88558","#f88457","#f88356","#f78155","#f78055","#f77f54","#f67d53","#f67c52","#f67b52","#f57951","#f57850","#f4774f","#f4754f","#f4744e","#f3734d","#f3714c","#f2704c","#f26f4b","#f16d4a","#f16c49","#f06b49","#f06948","#ef6847","#ef6646","#ee6545","#ed6344","#ed6243","#ec6042","#ec5f42","#eb5d41","#ea5c40","#ea5a3f","#e9593e","#e8573c","#e8563b","#e7543a","#e65339","#e65138","#e55037","#e44e36","#e44c35","#e34b34","#e24932","#e14831","#e04630","#e0442f","#df432e","#de412d","#dd402b","#dc3e2a","#dc3c29","#db3b28","#da3927","#d93826","#d83624","#d73423","#d63322","#d53121","#d43020","#d32e1f","#d22c1e","#d12b1d","#d0291b","#cf281a","#ce2619","#cd2518","#cc2317","#cb2216","#ca2015","#c91f14","#c81d13","#c71c12","#c61b11","#c51911","#c31810","#c2170f","#c1150e","#c0140d","#bf130c","#be120c","#bc110b","#bb100a","#ba0e09","#b80d09","#b70c08","#b60b07","#b50b07","#b30a06","#b20906","#b10805","#af0705","#ae0704","#ac0604","#ab0504","#a90503","#a80403","#a60402","#a50302","#a40302","#a20302","#a00201","#9f0201","#9d0201","#9c0101","#9a0101","#990101","#970101","#960100","#940100","#920000","#910000","#8f0000","#8e0000","#8c0000","#8a0000","#890000","#870000","#860000","#840000","#820000","#810000","#7f0000"]
justRed = ["#ff0000"]  # typical bright red bounding box
time_color = OrRd  # Note: we want the last ind to be the color of the most recent frame & the 0th ind to be the furthest

def box_color(time_from_present, max_time=5):  # assumed seconds
    """ 
    input: box's time from the present moment, how far in the past boxes should show up
    output: [r, g, b, a] value of the box [0,255]
    """
    if time_from_present >= max_time:
        return [0,0,0,0]
    color_factor = (1 - time_from_present/max_time)**2  # get the squared dist from max time by normalizing [0,1]
    hex_color = time_color[round(color_factor * (len(time_color)-1))]
    return hex_to_rgb(hex_color) + [round(color_factor*255)]  # add alpha scaler to return

def hex_to_rgb(hex_str):
    """ helper to convert rgb hex to its individual values in a list 
    ref: https://stackoverflow.com/questions/29643352/converting-hex-to-rgb-value-in-python """
    h = hex_str.lstrip('#')
    return list(int(h[i:i+2], 16) for i in (0, 2, 4))


print("Example color conversion:\n", time_color[0], "=>", hex_to_rgb(time_color[0]))
print("Example Bounding Box colors (max_time=5, index=sec_passed):\n", [box_color(i) for i in range(6)])

In [ ]:
""" Reasources:
- highlighting an area - https://stackoverflow.com/questions/56472024/how-to-change-the-opacity-of-boxes-cv2-rectangle
- cv2 draw functions - https://docs.opencv.org/4.x/dc/da5/tutorial_py_drawing_functions.html
"""

rect_size = 2  # size of boxes that denote corners
def draw_box_corners(image, x,y, w,h, color, thickness):
    # TODO: figure out if x,y is top left or bottom left corner
    image = cv2.rectangle(image, (x,y), (rect_size, rect_size), color, thickness)
    image = cv2.rectangle(image, (x+w,y), (rect_size, rect_size), color, thickness)
    image = cv2.rectangle(image, (x,y-h), (rect_size, rect_size), color, thickness)
    return cv2.rectangle(image, (x+w,y-h), (rect_size, rect_size), color, thickness)


line_len = 4;
def draw_box_frame(image, x,y, w,h, color, thickness):
    # TODO: tweak positioning
    image = cv2.line(image, (x,y), (x+line_len, y), color, thickness)
    image = cv2.line(image, (x,y), (x, y-line_len), color, thickness)
    
    image = cv2.line(image, (x+w-line_len,y), (x+w,y), color, thickness)
    image = cv2.line(image, (x+w,y), (x+w,y-line_len), color, thickness)
    
    image = cv2.line(image, (x,y-h), (x+line_len, y-h), color, thickness)
    image = cv2.line(image, (x,y-h+line_len), (x, y-h), color, thickness)
    
    image = cv2.line(image, (x+w-line_len,y-h), (x+w, y-h), color, thickness)
    return cv2.line(image, (x+w,y-h+line_len), (x+w, y-h), color, thickness)
    
    
def draw_box(image, x,y, w,h, color, thickness):
    return cv2.rectangle(image, (x,y), (w,h), color, thickness)

In [ ]:
index = next((i for i, obj in enumerate(detic_dict) if obj['timestamp'] == frames_timestamps_arr[500]), -1)

#actually draw boxes on
for j in range(len(main_frame_arr)):  # for each frame
  color = (0, 0, 255)
  thickness = 2
    
#   for time_since_present in range(1, j+1):  # draw boxes for past images
#     image = draw_box(image, x, y, w, h, box_color(time_since_present), thickness)
    
  # draw boxes for present
  image = main_frame_arr[j]
  index = next((i for i, obj in enumerate(detic_dict) if obj['timestamp'] == frames_timestamps_arr[j]), -1)
  for i in range(len(detic_dict[index]["values"])):  # for each detected object?
    x = int(detic_dict[index]["values"][i]["xyxyn"][0] * 760)
    w = int(detic_dict[index]["values"][i]["xyxyn"][2] * 760)
    y = int(detic_dict[index]["values"][i]["xyxyn"][1] * 428)
    h = int(detic_dict[index]["values"][i]["xyxyn"][3] * 428)
    start_point = (x, y)
    end_point = (w, h)
    # draw object bounding box
    image = cv2.rectangle(image, start_point, end_point, color, thickness)
#     image = draw_box(image, x, y, w, h, box_color(j), thickness)

    # Draw red background rectangle
    image = cv2.rectangle(image, (x, y-15), (x + (w - x), y), (0,0,255), -1)  # TODO: figure this out

    # Add text
    image = cv2.putText(image, detic_dict[index]["values"][i]["label"], (x + 2,y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255,255,255), 1)